In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import warnings
warnings.filterwarnings('ignore')
import dtale

In [2]:
batting = pd.read_csv("Batting.csv")
fielding = pd.read_csv("Fielding.csv")
pitching = pd.read_csv("Pitching.csv")

In [3]:
batting

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110490,zimmejo02,2021,1,MIL,NL,2,1,0,0,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
110491,zimmeky01,2021,1,KCA,AL,52,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
110492,zimmery01,2021,1,WAS,NL,110,255,27,62,16,...,46.0,0.0,0.0,16,77.0,0.0,0.0,0.0,2.0,9.0
110493,zuberty01,2021,1,KCA,AL,31,1,0,0,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0


In [6]:
batting.dtypes

playerID     object
yearID        int64
stint         int64
teamID       object
lgID         object
G             int64
AB            int64
R             int64
H             int64
2B            int64
3B            int64
HR            int64
RBI         float64
SB          float64
CS          float64
BB            int64
SO          float64
IBB         float64
HBP         float64
SH          float64
SF          float64
GIDP        float64
dtype: object

In [10]:
aggregation_functions_b = {'G': 'sum','AB': 'sum', 'R':'sum','H':'sum','2B':'sum','3B':'sum','HR':'sum','RBI':'sum','SB':'sum','CS':'sum','BB':'sum','SO':'sum','IBB':'sum','HBP':'sum','SH':'sum','SF':'sum','GIDP':'sum'}
combined_b = batting.groupby(fielding['playerID']).aggregate(aggregation_functions_b)

In [11]:
combined_b.isna().sum()

G       0
AB      0
R       0
H       0
2B      0
3B      0
HR      0
RBI     0
SB      0
CS      0
BB      0
SO      0
IBB     0
HBP     0
SH      0
SF      0
GIDP    0
dtype: int64

In [12]:
combined_b

,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,
aaronha01,2256,6349,913,1718,289,36,166,794.0,188.0,84.0,738,980.0,60.0,51.0,55.0,45.0,128.0
aaronto01,1035,2955,424,802,122,20,66,342.0,75.0,32.0,343,507.0,28.0,8.0,32.0,12.0,65.0
aasedo01,741,1628,225,427,71,16,52,222.0,35.0,12.0,128,349.0,5.0,21.0,18.0,10.0,40.0
abadan01,5,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
abadijo01,102,419,53,108,21,6,4,40.0,0.0,0.0,2,38.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zupcibo01,229,53,4,9,0,0,0,2.0,2.0,0.0,6,8.0,0.0,0.0,0.0,0.0,1.0
zupofr01,223,707,86,228,43,5,8,87.0,29.0,19.0,49,59.0,6.0,2.0,3.0,7.0,12.0
zuvelpa01,733,1590,208,425,81,4,47,233.0,6.0,4.0,147,316.0,10.0,21.0,12.0,17.0,42.0


In [13]:
aggregation_functions_f = {'POS': 'first', 'G': 'sum','GS': 'sum', 'InnOuts':'sum','PO':'sum','A':'sum','E':'sum','DP':'sum'}
combined_f = fielding.groupby(fielding['playerID']).aggregate(aggregation_functions_f)

In [14]:
combined_f

,POS,G,GS,InnOuts,PO,A,E,DP
playerID,,,,,,,,
aardsda01,P,331,0.0,1011.0,11,29,3.0,2
aaronha01,OF,3020,2977.0,78414.0,7436,429,144.0,218
aaronto01,1B,387,206.0,6472.0,1317,113,22.0,124
aasedo01,P,448,91.0,3328.0,67,135,13.0,10
abadan01,1B,9,4.0,138.0,37,1,1.0,3
...,...,...,...,...,...,...,...,...
zupofr01,C,13,1.0,114.0,31,1,2.0,0
zuvelpa01,SS,201,136.0,3844.0,267,415,23.0,84
zuverge01,P,265,31.0,1847.0,45,145,7.0,10


In [26]:
merged_df = pd.merge(combined_f,combined_b, on='playerID',how = 'outer')

In [27]:
merged_df = merged_df.fillna(0)

In [28]:
merged_df

,POS,G_x,GS,InnOuts,PO,A,E,DP,G_y,AB,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,,,,
aardsda01,P,331,0.0,1011.0,11,29,3.0,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaronha01,OF,3020,2977.0,78414.0,7436,429,144.0,218,2256.0,6349.0,...,794.0,188.0,84.0,738.0,980.0,60.0,51.0,55.0,45.0,128.0
aaronto01,1B,387,206.0,6472.0,1317,113,22.0,124,1035.0,2955.0,...,342.0,75.0,32.0,343.0,507.0,28.0,8.0,32.0,12.0,65.0
aasedo01,P,448,91.0,3328.0,67,135,13.0,10,741.0,1628.0,...,222.0,35.0,12.0,128.0,349.0,5.0,21.0,18.0,10.0,40.0
abadan01,1B,9,4.0,138.0,37,1,1.0,3,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zupofr01,C,13,1.0,114.0,31,1,2.0,0,223.0,707.0,...,87.0,29.0,19.0,49.0,59.0,6.0,2.0,3.0,7.0,12.0
zuvelpa01,SS,201,136.0,3844.0,267,415,23.0,84,733.0,1590.0,...,233.0,6.0,4.0,147.0,316.0,10.0,21.0,12.0,17.0,42.0
zuverge01,P,265,31.0,1847.0,45,145,7.0,10,483.0,1082.0,...,97.0,16.0,8.0,79.0,153.0,8.0,7.0,10.0,7.0,19.0
